In [ ]:
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# Práctica: Modelo climático 1D de balance energético (albedo suave)

En este cuaderno se utiliza un modelo climático idealizado de balance energético unidimensional (1D), implementado con la librería **climlab**, para estudiar de forma conceptual cómo responde el sistema climático a un forzamiento radiativo externo.

El objetivo de la práctica **no es aprender programación en Python**, ni entender el funcionamiento interno de la librería, sino **interpretar los resultados físicos del modelo** y relacionarlos con los conceptos vistos en la asignatura.

A lo largo del cuaderno se ejecutará código que puede considerarse una *caja negra*.  
No es necesario modificarlo ni comprenderlo en detalle. En cada sección se indicará claramente **qué se está haciendo** y **qué debe observarse en los resultados**.


In [ ]:
## Preparación del entorno y carga de librerías

En esta sección se cargan las librerías necesarias y se configuran algunos parámetros generales del modelo.  
No es necesario modificar ni entender el contenido de la celda de código siguiente.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import climlab

print("climlab version:", climlab.__version__)

In [ ]:
# Celda 2 — Parámetros del modelo sencillo

# Horizonte temporal de integración (años)
NYEARS_SPINUP = 20

# Forzamiento radiativo (W m^-2)
# Se implementa como A -> A - F en la parametrización OLR = A + B*T
F = 0.0


In [ ]:
## Construcción del modelo climático

En esta sección se construye un modelo climático idealizado de balance energético unidimensional (1D), que representa la temperatura superficial media en función de la latitud y del momento del año.

El modelo incluye:
- radiación solar entrante dependiente de la latitud y de la estación;
- emisión infrarroja parametrizada de forma lineal;
- un término difusivo que representa el transporte meridional de energía.

En esta primera parte de la práctica, el albedo es fijo y no depende de la temperatura, por lo que el sistema presenta un comportamiento suave y reversible.


In [ ]:
# Construcción del modelo climático EBM 1D estacional (albedo suave)
m = climlab.EBM_seasonal()


In [ ]:
## Ajuste inicial del modelo (spin-up)

Al iniciar el modelo desde una condición inicial arbitraria, la temperatura del sistema no representa todavía un estado climático realista. Es necesario integrar el modelo durante un cierto tiempo para que la solución pierda la dependencia de las condiciones iniciales y alcance un régimen estacionario.

Este periodo de ajuste se denomina **spin-up**.  
Durante el spin-up, el modelo evoluciona libremente hasta que las variables climáticas muestran un comportamiento estable y repetible (por ejemplo, un ciclo estacional que se repite año tras año).

Solo una vez completado el spin-up se analizan e interpretan los resultados del modelo.


In [ ]:
# Ajuste inicial del modelo (spin-up)
# El modelo se integra durante varios años para eliminar la dependencia
# de la condición inicial y alcanzar un régimen climático estable.

for _ in range(NYEARS_SPINUP):
    m.integrate_years(1)


In [ ]:
# Coordenada de latitud
lat = m.state['Ts'].domain.axes['lat'].points

# Paso temporal del modelo (en días)
dt_days = m.timestep / 86400.0
nsteps_year = int(round(365.0 / dt_days))

# Almacenamos la evolución de Ts durante un año completo
Ts_year = np.zeros((len(lat), nsteps_year))

for k in range(nsteps_year):
    m.step_forward()
    Ts_year[:, k] = np.array(m.state['Ts']).ravel()

# Temperatura media anual en cada latitud
Ts_ann = Ts_year.mean(axis=1)

# Eje temporal (días del año)
time_days = np.arange(nsteps_year) * dt_days


In [ ]:
plt.figure(figsize=(6,4))
plt.plot(lat, Ts_ann)
plt.xlabel("Latitud (°)")
plt.ylabel("Temperatura superficial media anual Ts (°C)")
plt.title("Perfil latitudinal de la temperatura media anual")
plt.grid(True)
plt.show()


In [ ]:
# Latitudes de referencia para el ciclo estacional
target_lats = [0, 45, 75]
idx = [int(np.argmin(np.abs(lat - x))) for x in target_lats]

plt.figure(figsize=(6,4))
for x, i in zip(target_lats, idx):
    plt.plot(time_days, Ts_year[i, :], label=f"{lat[i]:.0f}°")

plt.xlabel("Día del año")
plt.ylabel("Temperatura superficial Ts (°C)")
plt.title("Ciclo estacional de la temperatura en distintas latitudes")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
## Respuesta del modelo a un forzamiento radiativo externo

En esta sección se estudia cómo responde el modelo climático a un **forzamiento radiativo externo**.

En el contexto de este modelo, un forzamiento radiativo representa una modificación externa del balance energético del sistema climático (por ejemplo, debida a cambios en la concentración de gases de efecto invernadero).

El forzamiento se introduce modificando el término de radiación infrarroja emitida (OLR), de modo que el sistema recibe, en promedio, más energía de la que emite. Tras aplicar el forzamiento, el modelo se integra de nuevo hasta alcanzar un nuevo estado estacionario.


In [ ]:
# Aplicación del forzamiento radiativo
# El forzamiento F se implementa como una reducción del parámetro A
# en la parametrización OLR = A + B*T

m.subprocess['LW'].A -= F


In [ ]:
Tras introducir el forzamiento radiativo, el modelo deja de estar en equilibrio.  
Es necesario integrar de nuevo el sistema durante un cierto tiempo para que alcance un nuevo estado estacionario coherente con el forzamiento aplicado.


In [ ]:
# Nuevo periodo de ajuste tras aplicar el forzamiento radiativo

for _ in range(NYEARS_SPINUP):
    m.integrate_years(1)


In [ ]:
# Almacenamos la evolución de Ts durante un año completo tras el forzamiento
Ts_year_F = np.zeros((len(lat), nsteps_year))

for k in range(nsteps_year):
    m.step_forward()
    Ts_year_F[:, k] = np.array(m.state['Ts']).ravel()

# Temperatura media anual tras el forzamiento
Ts_ann_F = Ts_year_F.mean(axis=1)


In [ ]:
plt.figure(figsize=(6,4))
plt.plot(lat, Ts_ann, label="Clima base (F = 0)")
plt.plot(lat, Ts_ann_F, label=f"Forzamiento aplicado (F = {F} W m$^{{-2}}$)")
plt.xlabel("Latitud (°)")
plt.ylabel("Temperatura superficial media anual Ts (°C)")
plt.title("Efecto del forzamiento radiativo en la temperatura media anual")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.figure(figsize=(6,4))
for x, i in zip(target_lats, idx):
    plt.plot(time_days, Ts_year[i, :], linestyle="--", label=f"{lat[i]:.0f}° (base)")
    plt.plot(time_days, Ts_year_F[i, :], label=f"{lat[i]:.0f}° (forzado)")

plt.xlabel("Día del año")
plt.ylabel("Temperatura superficial Ts (°C)")
plt.title("Comparación del ciclo estacional con y sin forzamiento radiativo")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
Al comparar el clima base con el clima forzado se observa que:

- la temperatura aumenta en todas las latitudes;
- el calentamiento no es uniforme, siendo diferente según la latitud;
- el modelo alcanza un nuevo estado estacionario tras el periodo de ajuste;
- para este modelo con albedo fijo, la respuesta es suave y no depende de la historia previa del sistema.

Este comportamiento cambiará al introducir un feedback adicional en el siguiente cuaderno de la práctica.


In [ ]:
## Resumen de resultados

En este cuaderno se ha trabajado con un modelo climático sencillo que presenta una respuesta suave y reversible al forzamiento radiativo externo.

Se ha observado que:
- el modelo alcanza un estado estacionario tras un periodo de spin-up;
- un forzamiento radiativo positivo desplaza la temperatura hacia valores más altos;
- la respuesta del sistema es aproximadamente monótona y no depende de las condiciones iniciales.

En el siguiente cuaderno de la práctica se introducirá un feedback adicional que modificará de forma cualitativa este comportamiento, dando lugar a respuestas no lineales y a la posible dependencia de la historia del sistema.
